In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps

from config import api_key

gmaps.configure(api_key=api_key)

In [2]:
city_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
city_data.head()

,Unnamed: 0,Date,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,1600804868,hobart,AU,-42.88,147.33,50.00,71,90,17.22
1,1,1600805035,jiangyou,CN,31.77,104.72,62.65,86,88,0.85
2,2,1600804328,new norfolk,AU,-42.78,147.06,46.99,78,77,1.99
3,3,1600805002,ushuaia,AR,-54.80,-68.30,44.60,70,20,17.22
4,4,1600805035,abu samrah,SY,35.30,37.18,69.08,80,0,9.19


In [3]:
locations = city_data[["Lat", 'Long']]

humidity = city_data['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
filtered_data = city_data.loc[city_data['Max Temp'] <= 80.00]
filtered_data = city_data.loc[city_data['Max Temp'] > 75.00]
filtered_data = city_data.loc[city_data['Cloudiness'] < 10]
filtered_data = city_data.loc[city_data['Wind Speed'] < 50]
filtered_data = city_data.loc[city_data['Humidity'] < 15]

filtered_data

,Unnamed: 0,Date,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
248,248,1600804369,najran,SA,17.49,44.13,84.20,12,72,4.70
280,280,1600805077,adrar,MR,20.50,-10.07,100.40,13,28,7.34
356,356,1600805089,tianpeng,CN,30.99,103.94,70.00,14,93,2.24
402,402,1600805097,bilma,NE,18.69,12.92,92.48,13,0,6.62
416,416,1600805099,rio cuarto,AR,-33.13,-64.35,80.60,12,23,20.80
442,442,1600805106,mandali,IQ,33.75,45.55,90.32,13,0,8.97
509,509,1600804922,maneadero,MX,31.72,-116.57,91.00,11,7,8.99
522,522,1600805119,abha,SA,18.22,42.51,75.20,13,3,7.02
552,552,1600804563,taoudenni,ML,22.68,-3.98,101.88,8,0,6.98


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_data.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,Unnamed: 0,Date,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
248,248,1600804369,najran,SA,17.49,44.13,84.20,12,72,4.70,
280,280,1600805077,adrar,MR,20.50,-10.07,100.40,13,28,7.34,
356,356,1600805089,tianpeng,CN,30.99,103.94,70.00,14,93,2.24,
402,402,1600805097,bilma,NE,18.69,12.92,92.48,13,0,6.62,
416,416,1600805099,rio cuarto,AR,-33.13,-64.35,80.60,12,23,20.80,
442,442,1600805106,mandali,IQ,33.75,45.55,90.32,13,0,8.97,
509,509,1600804922,maneadero,MX,31.72,-116.57,91.00,11,7,8.99,
522,522,1600805119,abha,SA,18.22,42.51,75.20,13,3,7.02,
552,552,1600804563,taoudenni,ML,22.68,-3.98,101.88,8,0,6.98,


In [6]:
params = {"radius": 5000, "types": "lodging", "key": api_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    responce = requests.get(base_url, params=params)
    
    hotel = responce.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Cannot find... skipping.")

hotel_df

Cannot find... skipping.
Cannot find... skipping.


,Unnamed: 0,Date,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
248,248,1600804369,najran,SA,17.49,44.13,84.20,12,72,4.70,محمد هادي ال فطيح
280,280,1600805077,adrar,MR,20.50,-10.07,100.40,13,28,7.34,
356,356,1600805089,tianpeng,CN,30.99,103.94,70.00,14,93,2.24,Shuangliu Love Home Business Hotel
402,402,1600805097,bilma,NE,18.69,12.92,92.48,13,0,6.62,Expeditions Ténére voyages
416,416,1600805099,rio cuarto,AR,-33.13,-64.35,80.60,12,23,20.80,Hotel Menossi
442,442,1600805106,mandali,IQ,33.75,45.55,90.32,13,0,8.97,بيت خالد امين
509,509,1600804922,maneadero,MX,31.72,-116.57,91.00,11,7,8.99,Casa De Thompson
522,522,1600805119,abha,SA,18.22,42.51,75.20,13,3,7.02,Abha Hotel
552,552,1600804563,taoudenni,ML,22.68,-3.98,101.88,8,0,6.98,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [8]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))